In [2]:
# Mount Google Drive
"""
No need to execute this block when working on local system.
"""
from google.colab import drive
drive.mount("/content/vdrive", force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/vdrive/


In [0]:
# Files to process
"""
Modify the locations below as per your directory struture.
"""
sp_dir = "/content/vdrive/My Drive/Colab Notebooks/Projects/Bondai/SP 500/"
root_dir = "/content/vdrive/My Drive/Colab Notebooks/Projects/Bondai/SP 500/data/"
data_dir = "/content/vdrive/My Drive/Colab Notebooks/Projects/Bondai/SP 500/data/raw/"
prep_dir = "/content/vdrive/My Drive/Colab Notebooks/Projects/Bondai/SP 500/data/prep/"
model_dir = "/content/vdrive/My Drive/Colab Notebooks/Projects/Bondai/SP 500/data/model-1/"

In [0]:
# Loading the csv tickers, train_tickers and test_tickers file
import pandas as pd
ticker_list_df = pd.read_csv(root_dir + "ticker_list.csv", header = None, names = ["Tickers"])
train_tickers_df = pd.read_csv(root_dir + "train_tickers.csv", header = None, names = ["Train Tickers"])
test_tickers_df = pd.read_csv(root_dir + "test_tickers.csv", header = None, names = ["Test Tickers"])

In [0]:
def get_actual_data(ticker):
    # INCOME STATEMENT
    income_statement = pd.read_excel(data_dir + ticker + ".xlsx", sheet_name = "income_statement")
    income_statement = income_statement.loc[["Gross Profit", "Operating Income", "Net Income"]]
    income_statement.rename(index = {
        "Gross Profit": "gross_profit",
        "Operating Income": "op_income",
        "Net Income": "net_income"
    }, inplace = True)
    
    # BALANCE SHEET
    balance_sheet = pd.read_excel(data_dir + ticker + ".xlsx", sheet_name = "balance_sheet")
    balance_sheet = balance_sheet.loc[["Total current assets", "Total non-current assets", "Total current liabilities", "Total non-current liabilities"]]
    balance_sheet.rename(index = {
        "Total current assets": "crr_asst",
        "Total non-current assets": "ncrr_asst",
        "Total current liabilities": "crr_libt",
        "Total non-current liabilities": "ncrr_libt"
    }, inplace = True)
    
    df = pd.concat([income_statement, balance_sheet])
    df = df[df.columns[::-1]]
    df = df.transpose()
    df = df[["net_income", "op_income", "gross_profit", "crr_asst", "ncrr_asst", "crr_libt", "ncrr_libt"]][-1:]
    df["Ticker"] = ticker
    df.set_index("Ticker", inplace = True)
#     print(df)
    return df

In [0]:
# the companies for which balance sheet without current and non-current assets and liabilities
def get_actual_data_2(ticker):
    # INCOME STATEMENT
    income_statement = pd.read_excel(data_dir + ticker + ".xlsx", sheet_name = "income_statement")
    income_statement = income_statement.loc[["Gross Profit", "Operating Income", "Net Income"]]
    income_statement.rename(index = {
        "Gross Profit": "gross_profit",
        "Operating Income": "op_income",
        "Net Income": "net_income"
    }, inplace = True)
    
    # BALANCE SHEET
    balance_sheet = pd.read_excel(data_dir + ticker + ".xlsx", sheet_name = "balance_sheet")
    balance_sheet = balance_sheet.loc[["Total assets", "Total liabilities"]]
    balance_sheet.rename(index = {
        "Total assets": "ncrr_asst",
        "Total liabilities": "ncrr_libt"
    }, inplace = True)
    
    df = pd.concat([income_statement, balance_sheet])
    df = df[df.columns[::-1]]
    df = df.transpose()
    df["Ticker"] = ticker
    df.set_index("Ticker", inplace = True)
    df.insert(0, "crr_asst", 0)
    df.insert(0, "crr_libt", 0)
    df = df[["net_income", "op_income", "gross_profit", "crr_asst", "ncrr_asst", "crr_libt", "ncrr_libt"]][-1:]
    return df

In [100]:
df = pd.DataFrame(columns = ["net_income", "op_income", "gross_profit", "crr_asst", "ncrr_asst", "crr_libt", "ncrr_libt"])
df.index.name = "Ticker"
counter = 0
for ticker in test_tickers_df["Test Tickers"]:
    counter += 1
    print(str(counter) + ". Getting data for: " + ticker)
    try:
        df = df.append(get_actual_data(ticker))
    except:
        df = df.append(get_actual_data_2(ticker))

1. Getting data for: HSIC
2. Getting data for: ALXN
3. Getting data for: KR
4. Getting data for: BBT
5. Getting data for: DIS
6. Getting data for: MMC
7. Getting data for: MAR
8. Getting data for: CELG
9. Getting data for: VMC
10. Getting data for: RHI
11. Getting data for: MKC
12. Getting data for: PGR
13. Getting data for: ILMN
14. Getting data for: AMAT
15. Getting data for: LEG
16. Getting data for: STI
17. Getting data for: GS
18. Getting data for: RTN
19. Getting data for: KMI
20. Getting data for: CRM
21. Getting data for: STZ
22. Getting data for: INFO
23. Getting data for: AAP
24. Getting data for: EXC
25. Getting data for: MCO
26. Getting data for: AGN
27. Getting data for: C
28. Getting data for: NTRS
29. Getting data for: TMK
30. Getting data for: EW
31. Getting data for: NEE
32. Getting data for: BBY
33. Getting data for: RCL
34. Getting data for: VIAB
35. Getting data for: TTWO
36. Getting data for: AWK
37. Getting data for: MO
38. Getting data for: WAT
39. Getting data f

In [104]:
df = df.dropna()
df

,net_income,op_income,gross_profit,crr_asst,ncrr_asst,crr_libt,ncrr_libt
Ticker,,,,,,,
HSIC,5.358810e+08,7.530520e+08,3.595084e+09,4.175220e+09,4.325307e+09,3.218827e+09,1.427756e+09
ALXN,7.760000e+07,3.861000e+08,3.756900e+09,3.385000e+09,1.054690e+10,1.174000e+09,3.592600e+09
KR,1.907000e+09,2.085000e+09,2.700000e+10,1.111700e+10,2.608000e+10,1.419700e+10,1.609500e+10
BBT,3.237000e+09,4.060000e+09,1.099200e+10,0.000000e+00,2.256970e+11,0.000000e+00,1.955190e+11
DIS,1.259800e+10,1.480400e+10,2.670800e+10,1.682500e+10,8.177300e+10,1.786000e+10,2.790600e+10
MMC,1.650000e+09,2.761000e+09,1.495000e+10,5.934000e+09,1.564400e+10,4.924000e+09,9.070000e+09
MAR,1.907000e+09,2.366000e+09,3.674000e+09,2.706000e+09,2.099000e+10,6.437000e+09,1.503400e+10
CELG,4.046000e+09,5.191000e+09,1.469400e+10,9.067000e+09,2.641300e+10,4.057000e+09,2.526200e+10
VMC,5.158050e+08,7.477130e+08,1.100945e+09,1.079145e+09,8.752985e+09,6.025500e+08,4.026677e+09


In [0]:
df.to_csv(model_dir + "actual_data.csv")

In [0]:
def get_prev_data(ticker):
    # INCOME STATEMENT
    income_statement = pd.read_excel(data_dir + ticker + ".xlsx", sheet_name = "income_statement")
    income_statement = income_statement.loc[["Gross Profit", "Operating Income", "Net Income"]]
    income_statement.rename(index = {
        "Gross Profit": "gross_profit",
        "Operating Income": "op_income",
        "Net Income": "net_income"
    }, inplace = True)
    
    # BALANCE SHEET
    balance_sheet = pd.read_excel(data_dir + ticker + ".xlsx", sheet_name = "balance_sheet")
    balance_sheet = balance_sheet.loc[["Total current assets", "Total non-current assets", "Total current liabilities", "Total non-current liabilities"]]
    balance_sheet.rename(index = {
        "Total current assets": "crr_asst",
        "Total non-current assets": "ncrr_asst",
        "Total current liabilities": "crr_libt",
        "Total non-current liabilities": "ncrr_libt"
    }, inplace = True)
    
    df = pd.concat([income_statement, balance_sheet])
    df = df[df.columns[::-1]]
    df = df.transpose()
    df = df[["net_income", "op_income", "gross_profit", "crr_asst", "ncrr_asst", "crr_libt", "ncrr_libt"]][-2:-1]
    df["Ticker"] = ticker
    df.set_index("Ticker", inplace = True)
#     print(df)
    return df

# get_prev_data("HSIC")

In [0]:
# the companies for which balance sheet without current and non-current assets and liabilities
def get_prev_data_2(ticker):
    # INCOME STATEMENT
    income_statement = pd.read_excel(data_dir + ticker + ".xlsx", sheet_name = "income_statement")
    income_statement = income_statement.loc[["Gross Profit", "Operating Income", "Net Income"]]
    income_statement.rename(index = {
        "Gross Profit": "gross_profit",
        "Operating Income": "op_income",
        "Net Income": "net_income"
    }, inplace = True)
    
    # BALANCE SHEET
    balance_sheet = pd.read_excel(data_dir + ticker + ".xlsx", sheet_name = "balance_sheet")
    balance_sheet = balance_sheet.loc[["Total assets", "Total liabilities"]]
    balance_sheet.rename(index = {
        "Total assets": "ncrr_asst",
        "Total liabilities": "ncrr_libt"
    }, inplace = True)
    
    df = pd.concat([income_statement, balance_sheet])
    df = df[df.columns[::-1]]
    df = df.transpose()
    df["Ticker"] = ticker
    df.set_index("Ticker", inplace = True)
    df.insert(0, "crr_asst", 0)
    df.insert(0, "crr_libt", 0)
    df = df[["net_income", "op_income", "gross_profit", "crr_asst", "ncrr_asst", "crr_libt", "ncrr_libt"]][-2:-1]
    return df

In [121]:
df2 = pd.DataFrame(columns = ["net_income", "op_income", "gross_profit", "crr_asst", "ncrr_asst", "crr_libt", "ncrr_libt"])
df2.index.name = "Ticker"
counter = 0
for ticker in test_tickers_df["Test Tickers"]:
    counter += 1
    print(str(counter) + ". Getting data for: " + ticker)
    try:
        df2 = df2.append(get_prev_data(ticker))
    except:
        df2 = df2.append(get_prev_data_2(ticker))

1. Getting data for: HSIC
2. Getting data for: ALXN
3. Getting data for: KR
4. Getting data for: BBT
5. Getting data for: DIS
6. Getting data for: MMC
7. Getting data for: MAR
8. Getting data for: CELG
9. Getting data for: VMC
10. Getting data for: RHI
11. Getting data for: MKC
12. Getting data for: PGR
13. Getting data for: ILMN
14. Getting data for: AMAT
15. Getting data for: LEG
16. Getting data for: STI
17. Getting data for: GS
18. Getting data for: RTN
19. Getting data for: KMI
20. Getting data for: CRM
21. Getting data for: STZ
22. Getting data for: INFO
23. Getting data for: AAP
24. Getting data for: EXC
25. Getting data for: MCO
26. Getting data for: AGN
27. Getting data for: C
28. Getting data for: NTRS
29. Getting data for: TMK
30. Getting data for: EW
31. Getting data for: NEE
32. Getting data for: BBY
33. Getting data for: RCL
34. Getting data for: VIAB
35. Getting data for: TTWO
36. Getting data for: AWK
37. Getting data for: MO
38. Getting data for: WAT
39. Getting data f

In [122]:
df2 = df2.dropna()
df2

,net_income,op_income,gross_profit,crr_asst,ncrr_asst,crr_libt,ncrr_libt
Ticker,,,,,,,
HSIC,4.062990e+08,8.593690e+08,3.399103e+09,4.086020e+09,3.777975e+09,2.828975e+09,1.378472e+09
ALXN,4.433000e+08,6.684000e+08,3.096900e+09,2.953900e+09,1.062940e+10,9.525000e+08,3.737700e+09
KR,1.975000e+09,3.436000e+09,2.583500e+10,1.034000e+10,2.616500e+10,1.286000e+10,1.693500e+10
BBT,2.394000e+09,3.718000e+09,1.077000e+10,0.000000e+00,2.216420e+11,0.000000e+00,1.919470e+11
DIS,8.980000e+09,1.377500e+10,2.483100e+10,1.588900e+10,7.990000e+10,1.959500e+10,3.119000e+10
MMC,1.492000e+09,2.655000e+09,1.402400e+10,5.562000e+09,1.486700e+10,4.262000e+09,8.725000e+09
MAR,1.459000e+09,2.504000e+09,3.813000e+09,2.740000e+09,2.110600e+10,5.807000e+09,1.445700e+10
CELG,2.940000e+09,4.707000e+09,1.254200e+10,1.489200e+10,1.524900e+10,2.987000e+09,2.023300e+10
VMC,6.011850e+08,6.390440e+08,9.935130e+08,1.180101e+09,8.324790e+09,4.428720e+08,4.093126e+09


In [0]:
df2.to_csv(model_dir + "prev_data.csv")

In [0]:
def prepare_test_data(data, n_steps):
    for i in range(1, len(data)):
        end_ix = i + n_steps
        if(end_ix > len(data) - 1):
            seq_x = data[i-1:end_ix-1, :]
            return seq_x

In [0]:
def read_data_from_file(ticker):
    df = pd.read_csv(prep_dir + ticker + ".csv")
    return df[["net_income", "op_income", "gross_profit", "crr_asst", "ncrr_asst", "crr_libt", "ncrr_libt"]].values

In [0]:
# Load trained model
from keras.models import load_model
model = load_model(sp_dir + "bondai_model_1.1.h5")

In [178]:
df3 = pd.DataFrame(columns = ["net_income", "op_income", "gross_profit", "crr_asst", "ncrr_asst", "crr_libt", "ncrr_libt"])
df3.index.name = "Ticker"
test_df = pd.DataFrame(columns = ["net_income", "op_income", "gross_profit", "crr_asst", "ncrr_asst", "crr_libt", "ncrr_libt"])
test_df.index.name = "Ticker"
counter = 0
for ticker in test_tickers_df["Test Tickers"]:
    counter += 1
    print(str(counter) + ". Predicting values for: " + ticker)
    test_array = prepare_test_data(read_data_from_file(ticker), 2)
    test_input = test_array.reshape(1, 2, 7)
    pred_array = model.predict(test_input, verbose = 1)
    df_temp = pd.DataFrame(pred_array.tolist(), columns = ["net_income", "op_income", "gross_profit", "crr_asst", "ncrr_asst", "crr_libt", "ncrr_libt"])
    df_temp["Ticker"] = ticker
    df_temp.set_index("Ticker", inplace = True)
    test_df = test_df.append(df_temp)
    df_temp = df_temp.loc[ticker]*df2.loc[ticker]
    df3 = df3.append(df_temp)
    

1. Predicting values for: HSIC
1/1 [==============================] - 0s 293ms/step
2. Predicting values for: ALXN
1/1 [==============================] - 0s 1ms/step
3. Predicting values for: KR
1/1 [==============================] - 0s 2ms/step
4. Predicting values for: BBT
1/1 [==============================] - 0s 1ms/step
5. Predicting values for: DIS
1/1 [==============================] - 0s 2ms/step
6. Predicting values for: MMC
1/1 [==============================] - 0s 2ms/step
7. Predicting values for: MAR
1/1 [==============================] - 0s 3ms/step
8. Predicting values for: CELG
1/1 [==============================] - 0s 1ms/step
9. Predicting values for: VMC
1/1 [==============================] - 0s 5ms/step
10. Predicting values for: RHI
1/1 [==============================] - 0s 3ms/step
11. Predicting values for: MKC
1/1 [==============================] - 0s 1ms/step
12. Predicting values for: PGR
1/1 [==============================] - 0s 1ms/step
13. Predicting values

In [179]:
test_df

,net_income,op_income,gross_profit,crr_asst,ncrr_asst,crr_libt,ncrr_libt
Ticker,,,,,,,
HSIC,90.567238,35.086239,14.530129,3.199066,12.404232,9.485085,6.233299
ALXN,11.829369,29.504908,6.072799,5.710222,9.407384,7.486669,18.218327
KR,6.805795,20.842474,0.706779,6.696035,9.308869,8.750217,14.008575
BBT,4.631532,7.631013,4.076230,4.039822,6.841225,7.078929,6.557758
DIS,6.123345,-3.027328,3.794250,4.097010,6.743133,7.032441,6.772195
MMC,23.680965,13.961922,6.381871,4.081548,8.549265,7.774268,7.294129
MAR,278.247192,19.505434,38.074577,21.264244,39.012157,25.189312,-5.472239
CELG,2.759505,6.238966,4.803248,4.299804,6.992246,7.321340,6.957475
VMC,1.012426,1.713614,3.753136,3.853374,6.492170,6.870212,6.646871


In [180]:
df3

,net_income,op_income,gross_profit,crr_asst,ncrr_asst,crr_libt,ncrr_libt
Ticker,,,,,,,
HSIC,3.679738e+10,3.015203e+10,4.938941e+10,1.307145e+10,4.686288e+10,2.683307e+10,8.592428e+09
ALXN,5.243959e+09,1.972108e+10,1.880685e+10,1.686743e+10,9.999485e+10,7.131052e+09,6.809464e+10
KR,1.344144e+10,7.161474e+10,1.825964e+10,6.923701e+10,2.435666e+11,1.125278e+11,2.372352e+11
BBT,1.108789e+10,2.837211e+10,4.390100e+10,0.000000e+00,1.516303e+12,0.000000e+00,1.258742e+12
DIS,5.498764e+10,-4.170145e+10,9.421502e+10,6.509739e+10,5.387763e+11,1.378007e+11,2.112248e+11
MMC,3.533200e+10,3.706890e+10,8.949936e+10,2.270157e+10,1.271019e+11,3.313393e+10,6.364128e+10
MAR,4.059627e+11,4.884161e+10,1.451784e+11,5.826403e+10,8.233906e+11,1.462743e+11,-7.911217e+10
CELG,8.112943e+09,2.936682e+10,6.024234e+10,6.403268e+10,1.066248e+11,2.186884e+10,1.407706e+11
VMC,6.086551e+08,1.095075e+09,3.728789e+09,4.547371e+09,5.404595e+10,3.042625e+09,2.720648e+10


In [0]:
df3.to_csv(model_dir + "pred_data.csv")